Needs 3 text files with data, dataset.py, dataloader.py, and models (feedforward.py)

In [1]:
! pip install --quiet "torchvision" "torch>=1.6, <1.9" "torchmetrics>=0.3" "ipython[notebook]" "pytorch-lightning>=1.3" "torchtext"
! pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.6.1'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl (144.6 MB)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-librar

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/Othercomputers/My MacBook Pro/NLP/HW2/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from dataset import wiki_dataset
from dataloader import wiki_dataloader
import feedforward
import LSTM
import torchmetrics
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# test the feedforward across some different parameters (lr, dropout)
#feedforward.test_hparam('lr', values=[1e-4, 5e-4, 1e-3], tpu_cores=8, gpus=None)
#feedforward.test_hparam('dropout', values=[0.2, 0.5, 0.7], tpu_cores=8, gpus=None)

# test the LSTM across some different parameters (lr, dropout, gradient clipping)
LSTM.test_hparam('lr', values=[1e-4, 5e-4, 1e-3], tpu_cores=8, gpus=None)
#LSTM.test_hparam('dropout', values=[0.2, 0.5, 0.7], tpu_cores=8, gpus=None)
#LSTM.test_hparam('gradient_clip_val', values=[0.2, 0.5, 1.0], tpu_cores=8, gpus=None)


GPU available: False, used: False
TPU available: True, using: 8 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type             | Params
-------------------------------------------
0 | embed | Embedding        | 2.7 M 
1 | lin1  | Linear           | 50.1 K
2 | bn1   | BatchNorm1d      | 200   
3 | drop1 | Dropout          | 0     
4 | lin2  | Linear           | 2.8 M 
5 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.287    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

printing some example sentences from test set
--> format: sentence (true) [predicted]
the drum theatre in plymouth (and) [and]
the bill he had a (recurring) [<unk>]
frequently called the greatest of (the) [the]
as darren in the <year> (theatre) [<integer>]
film <unk> directed by paris (<unk>) [<unk>]
of the television series the (long) [<unk>]
in two films in <year> (<unk>) [the]
cast in the <year> theatre (productions) [was]
performed at bush theatre in (the) [the]
neil in <month> <year> he (had) [was]


GPU available: False, used: False
TPU available: True, using: 8 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "

  | Name  | Type             | Params
-------------------------------------------
0 | embed | Embedding        | 2.7 M 
1 | lin1  | Linear           | 50.1 K
2 | bn1   | BatchNorm1d      | 200   
3 | drop1 | Dropout          | 0     
4 | lin2  | Linear           | 2.8 M 
5 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.287    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]